# IPL Analysis

In [2]:
import sys
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.functions import split, explode

In [3]:
spark = SparkSession.builder.getOrCreate()
rawDF = spark.read.json("/Users/surajs/Desktop/Bigdata/data/335982.json", multiLine = "true")

In [4]:
rawDF.printSchema()

root
 |-- info: struct (nullable = true)
 |    |-- balls_per_over: long (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- dates: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- event: struct (nullable = true)
 |    |    |-- match_number: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- match_type: string (nullable = true)
 |    |-- officials: struct (nullable = true)
 |    |    |-- match_referees: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- reserve_umpires: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- tv_umpires: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- umpires: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- outcome: struct (nullable = true)
 |    |    |-

# First Innings

In [5]:
inningDF=rawDF.select(f.col('innings').getItem(0).alias('innings1'))

In [6]:
inningDF.show()

+--------------------+
|            innings1|
+--------------------+
|{[{[{SC Ganguly, ...|
+--------------------+



In [7]:
overDF=inningDF.select(explode('innings1.overs').alias('overs'))
overDF.show()

+--------------------+
|               overs|
+--------------------+
|{[{SC Ganguly, P ...|
|{[{BB McCullum, Z...|
|{[{SC Ganguly, P ...|
|{[{BB McCullum, A...|
|{[{SC Ganguly, P ...|
|{[{BB McCullum, Z...|
|{[{BB McCullum, A...|
|{[{BB McCullum, Z...|
|{[{BB McCullum, J...|
|{[{RT Ponting, SB...|
|{[{BB McCullum, J...|
|{[{BB McCullum, S...|
|{[{RT Ponting, JH...|
|{[{BB McCullum, S...|
|{[{DJ Hussey, CL ...|
|{[{DJ Hussey, AA ...|
|{[{BB McCullum, Z...|
|{[{DJ Hussey, AA ...|
|{[{BB McCullum, J...|
|{[{BB McCullum, P...|
+--------------------+



In [8]:
deliveryDF=overDF.select(explode('overs.deliveries').alias('deliveries'))
deliveryDF.show()

+--------------------+
|          deliveries|
+--------------------+
|{SC Ganguly, P Ku...|
|{BB McCullum, P K...|
|{BB McCullum, P K...|
|{BB McCullum, P K...|
|{BB McCullum, P K...|
|{BB McCullum, P K...|
|{BB McCullum, P K...|
|{BB McCullum, Z K...|
|{BB McCullum, Z K...|
|{BB McCullum, Z K...|
|{BB McCullum, Z K...|
|{BB McCullum, Z K...|
|{BB McCullum, Z K...|
|{SC Ganguly, P Ku...|
|{SC Ganguly, P Ku...|
|{SC Ganguly, P Ku...|
|{BB McCullum, P K...|
|{BB McCullum, P K...|
|{SC Ganguly, P Ku...|
|{BB McCullum, AA ...|
+--------------------+
only showing top 20 rows



In [9]:
runsDF=deliveryDF.select('deliveries.batter','deliveries.bowler',f.col('deliveries.runs.batter').alias("runs_scored"))
runsDF.show()

+-----------+---------+-----------+
|     batter|   bowler|runs_scored|
+-----------+---------+-----------+
| SC Ganguly|  P Kumar|          0|
|BB McCullum|  P Kumar|          0|
|BB McCullum|  P Kumar|          0|
|BB McCullum|  P Kumar|          0|
|BB McCullum|  P Kumar|          0|
|BB McCullum|  P Kumar|          0|
|BB McCullum|  P Kumar|          0|
|BB McCullum|   Z Khan|          0|
|BB McCullum|   Z Khan|          4|
|BB McCullum|   Z Khan|          4|
|BB McCullum|   Z Khan|          6|
|BB McCullum|   Z Khan|          4|
|BB McCullum|   Z Khan|          0|
| SC Ganguly|  P Kumar|          0|
| SC Ganguly|  P Kumar|          0|
| SC Ganguly|  P Kumar|          0|
|BB McCullum|  P Kumar|          4|
|BB McCullum|  P Kumar|          1|
| SC Ganguly|  P Kumar|          0|
|BB McCullum|AA Noffke|          0|
+-----------+---------+-----------+
only showing top 20 rows



In [10]:
totalrunDF=runsDF.groupBy('batter','bowler').agg(f.sum('runs_scored').alias('total_runs_scored'))
totalrunDF.show()

+---------------+---------+-----------------+
|         batter|   bowler|total_runs_scored|
+---------------+---------+-----------------+
|    BB McCullum|  P Kumar|               32|
|     RT Ponting|JH Kallis|               11|
|     RT Ponting| SB Joshi|                3|
|      DJ Hussey| CL White|                6|
|      DJ Hussey|   Z Khan|                3|
|      DJ Hussey| SB Joshi|                2|
|    BB McCullum|AA Noffke|               24|
|    BB McCullum| CL White|               16|
|     RT Ponting|   Z Khan|                2|
|    BB McCullum|   Z Khan|               33|
|     RT Ponting|AA Noffke|                4|
|    BB McCullum| SB Joshi|               21|
|Mohammad Hafeez|JH Kallis|                4|
|     SC Ganguly|  P Kumar|                5|
|Mohammad Hafeez|AA Noffke|                1|
|      DJ Hussey|AA Noffke|                1|
|     SC Ganguly|   Z Khan|                0|
|     SC Ganguly|AA Noffke|                5|
|    BB McCullum|JH Kallis|       

In [11]:
maxrunsDF=totalrunDF.groupBy('batter').agg(f.max('total_runs_scored').alias('max_runs_scored'))
maxrunsDF.show()

+---------------+---------------+
|         batter|max_runs_scored|
+---------------+---------------+
|Mohammad Hafeez|              4|
|     RT Ponting|             11|
|      DJ Hussey|              6|
|    BB McCullum|             33|
|     SC Ganguly|              5|
+---------------+---------------+



In [12]:
resDF=totalrunDF.join(maxrunsDF, (maxrunsDF.max_runs_scored==totalrunDF.total_runs_scored) & (maxrunsDF.batter==totalrunDF.batter), how="leftsemi").withColumnRenamed("total_runs_scored", "max_runs_scored")
resDF.show()

+---------------+---------+---------------+
|         batter|   bowler|max_runs_scored|
+---------------+---------+---------------+
|     RT Ponting|JH Kallis|             11|
|      DJ Hussey| CL White|              6|
|    BB McCullum|   Z Khan|             33|
|Mohammad Hafeez|JH Kallis|              4|
|     SC Ganguly|  P Kumar|              5|
|     SC Ganguly|AA Noffke|              5|
+---------------+---------+---------------+



# Second Innings

In [13]:
inningDF2=rawDF.select(f.col('innings').getItem(1).alias('innings2'))
inningDF2.show()

+--------------------+
|            innings2|
+--------------------+
|{[{[{R Dravid, AB...|
+--------------------+



In [14]:
overDF2=inningDF2.select(explode('innings2.overs').alias('overs'))
overDF2.show()

+--------------------+
|               overs|
+--------------------+
|{[{R Dravid, AB D...|
|{[{R Dravid, I Sh...|
|{[{V Kohli, AB Di...|
|{[{W Jaffer, I Sh...|
|{[{JH Kallis, AB ...|
|{[{W Jaffer, AB D...|
|{[{CL White, AB A...|
|{[{CL White, SC G...|
|{[{B Akhil, AB Ag...|
|{[{AA Noffke, SC ...|
|{[{AA Noffke, AB ...|
|{[{AA Noffke, SC ...|
|{[{P Kumar, LR Sh...|
|{[{Z Khan, SC Gan...|
|{[{SB Joshi, I Sh...|
|{[{SB Joshi, LR S...|
+--------------------+



In [15]:
deliveryDF2=overDF2.select(explode('overs.deliveries').alias('deliveries'))
deliveryDF2.show()

+--------------------+
|          deliveries|
+--------------------+
|{R Dravid, AB Din...|
|{W Jaffer, AB Din...|
|{W Jaffer, AB Din...|
|{W Jaffer, AB Din...|
|{R Dravid, AB Din...|
|{W Jaffer, AB Din...|
|{W Jaffer, AB Din...|
|{R Dravid, I Shar...|
|{V Kohli, I Sharm...|
|{V Kohli, I Sharm...|
|{V Kohli, I Sharm...|
|{W Jaffer, I Shar...|
|{W Jaffer, I Shar...|
|{V Kohli, AB Dind...|
|{V Kohli, AB Dind...|
|{JH Kallis, AB Di...|
|{W Jaffer, AB Din...|
|{W Jaffer, AB Din...|
|{W Jaffer, AB Din...|
|{W Jaffer, AB Din...|
+--------------------+
only showing top 20 rows



In [16]:
runsDF2=deliveryDF2.select('deliveries.batter','deliveries.bowler',f.col('deliveries.runs.batter').alias("runs_scored"))
runsDF2.show()

+---------+--------+-----------+
|   batter|  bowler|runs_scored|
+---------+--------+-----------+
| R Dravid|AB Dinda|          1|
| W Jaffer|AB Dinda|          0|
| W Jaffer|AB Dinda|          0|
| W Jaffer|AB Dinda|          1|
| R Dravid|AB Dinda|          1|
| W Jaffer|AB Dinda|          0|
| W Jaffer|AB Dinda|          0|
| R Dravid|I Sharma|          0|
|  V Kohli|I Sharma|          0|
|  V Kohli|I Sharma|          0|
|  V Kohli|I Sharma|          1|
| W Jaffer|I Sharma|          0|
| W Jaffer|I Sharma|          0|
|  V Kohli|AB Dinda|          0|
|  V Kohli|AB Dinda|          0|
|JH Kallis|AB Dinda|          1|
| W Jaffer|AB Dinda|          0|
| W Jaffer|AB Dinda|          0|
| W Jaffer|AB Dinda|          0|
| W Jaffer|AB Dinda|          1|
+---------+--------+-----------+
only showing top 20 rows



In [17]:
totalrunsDF2=runsDF2.groupBy('batter','bowler').agg(f.sum('runs_scored').alias('total_runs_scored'))
totalrunsDF2.show()

+----------+----------+-----------------+
|    batter|    bowler|total_runs_scored|
+----------+----------+-----------------+
|MV Boucher|AB Agarkar|                4|
|  W Jaffer|AB Agarkar|                1|
| AA Noffke|AB Agarkar|                2|
| AA Noffke|SC Ganguly|                7|
|   P Kumar| LR Shukla|                7|
|  SB Joshi|SC Ganguly|                1|
|   P Kumar|  I Sharma|                0|
|  W Jaffer|  AB Dinda|                2|
|  SB Joshi|  I Sharma|                2|
|  W Jaffer|  I Sharma|                3|
|  CL White|  AB Dinda|                1|
|   V Kohli|  I Sharma|                1|
|   B Akhil|AB Agarkar|                0|
|    Z Khan| LR Shukla|                2|
|   P Kumar|SC Ganguly|                7|
|   V Kohli|  AB Dinda|                0|
| JH Kallis|AB Agarkar|                7|
|  R Dravid|  AB Dinda|                2|
|    Z Khan|SC Ganguly|                1|
|  CL White|SC Ganguly|                2|
+----------+----------+-----------

In [18]:
maxrunsDF2=totalrunsDF2.groupBy('batter').agg(f.max('total_runs_scored').alias('max_runs_scored'))
maxrunsDF2.show()

+----------+---------------+
|    batter|max_runs_scored|
+----------+---------------+
|  SB Joshi|              2|
|  R Dravid|              2|
| JH Kallis|              7|
| AA Noffke|              7|
|  CL White|              3|
|  W Jaffer|              3|
|   B Akhil|              0|
|   V Kohli|              1|
|   P Kumar|              7|
|MV Boucher|              4|
|    Z Khan|              2|
+----------+---------------+



In [19]:
resDF2=totalrunsDF2.join(maxrunsDF2, (maxrunsDF2.max_runs_scored==totalrunsDF2.total_runs_scored) & (maxrunsDF2.batter==totalrunsDF2.batter), how="leftsemi").withColumnRenamed("total_runs_scored", "max_runs_scored")
resDF2.show()

+----------+----------+---------------+
|    batter|    bowler|max_runs_scored|
+----------+----------+---------------+
|MV Boucher|AB Agarkar|              4|
| AA Noffke|SC Ganguly|              7|
|   P Kumar| LR Shukla|              7|
|  SB Joshi|  I Sharma|              2|
|  W Jaffer|  I Sharma|              3|
|   V Kohli|  I Sharma|              1|
|   B Akhil|AB Agarkar|              0|
|    Z Khan| LR Shukla|              2|
|   P Kumar|SC Ganguly|              7|
| JH Kallis|AB Agarkar|              7|
|  R Dravid|  AB Dinda|              2|
|  CL White|AB Agarkar|              3|
+----------+----------+---------------+



In [21]:
minrunsDF2=totalrunsDF2.groupBy('batter').agg(f.min('total_runs_scored').alias('min_runs_scored'))
minrunsDF2.show()

+----------+---------------+
|    batter|min_runs_scored|
+----------+---------------+
|  SB Joshi|              0|
|  R Dravid|              0|
| JH Kallis|              0|
| AA Noffke|              2|
|  CL White|              1|
|  W Jaffer|              1|
|   B Akhil|              0|
|   V Kohli|              0|
|   P Kumar|              0|
|MV Boucher|              1|
|    Z Khan|              1|
+----------+---------------+



In [22]:
resuDF2=totalrunsDF2.join(minrunsDF2, (minrunsDF2.min_runs_scored==totalrunsDF2.total_runs_scored) & (minrunsDF2.batter==totalrunsDF2.batter), how="leftsemi").withColumnRenamed("total_runs_scored", "min_runs_scored")
resuDF2.show()

+----------+----------+---------------+
|    batter|    bowler|min_runs_scored|
+----------+----------+---------------+
|  W Jaffer|AB Agarkar|              1|
| AA Noffke|AB Agarkar|              2|
|   P Kumar|  I Sharma|              0|
|  CL White|  AB Dinda|              1|
|   B Akhil|AB Agarkar|              0|
|   V Kohli|  AB Dinda|              0|
|    Z Khan|SC Ganguly|              1|
|  SB Joshi| LR Shukla|              0|
|MV Boucher|  AB Dinda|              1|
|  R Dravid|  I Sharma|              0|
| JH Kallis|  I Sharma|              0|
+----------+----------+---------------+

